In [ ]:
# --- 0. ENVIRONMENT SETUP AND IMPORTS ---
# Note: Running pip install at the beginning
!pip install sentinelhub matplotlib pandas numpy geopandas osmnx rasterio scipy shapely --quiet

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import osmnx as ox
import rasterio
from rasterio.transform import from_bounds
from rasterio.features import rasterize
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
)
import os
import pandas as pd
from scipy.interpolate import RegularGridInterpolator
from rasterio.transform import Affine
from shapely.geometry import Polygon
from shapely.ops import unary_union # Added for geometry union
from shapely.geometry import Point

# --- 1. CONSTANTS AND PROJECT SETTINGS ---

OUTPUT_DIR = "race_publica_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"--- 1. Project Configuration ---")
print(f"✅ Created working directory: {OUTPUT_DIR}")

# AOI (Berlin): [West, South, East, North]
aoi_coords_wgs84 = [13.23812, 52.56918, 13.27125, 52.58078]
time_interval = ("2024-08-01", "2024-09-30")
resolution = 5 # meters per pixel
GEOJSON_WATERWAYS_FILENAME = "berlin_waterways.geojson"
# Changed filename for landmass
LAND_GEOJSON_FILENAME = "landmass_from_water_inverse.geojson"

aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

# --- 2. HELPER FUNCTIONS ---

# ... (plot_image, save_raster_data, plot_geojson_only - NO CHANGES) ...

def plot_image(image, factor=1.0, clip_range=None, title="True Color Image"):
    """Function to display raster images."""
    fig, ax = plt.subplots(figsize=(10, 10))
    if clip_range:
        ax.imshow(np.clip(image * factor, *clip_range))
    else:
        ax.imshow(image * factor)
    ax.set_title(title, fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

def save_raster_data(image, aoi_coords, aoi_size, filename):
    """Saves a NumPy array as a GeoTIFF."""
    lon_min, lat_min, lon_max, lat_max = aoi_coords
    width, height = aoi_size
    transform = from_bounds(lon_min, lat_min, lon_max, lat_max, width, height)
    count_val = image.shape[2] if image.ndim == 3 else 1

    with rasterio.open(
        os.path.join(OUTPUT_DIR, filename),
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=count_val,
        dtype=image.dtype,
        crs=CRS.WGS84.ogc_string(),
        transform=transform,
    ) as dst:
        if image.ndim == 3:
            for i in range(image.shape[2]):
                dst.write(image[:, :, i], i + 1)
        else:
            dst.write(image, 1)
    print(f"✅ Raster data saved as: {OUTPUT_DIR}/{filename}")
    return transform

def plot_overlay(raster_image, gdf_waterways=None, gdf_land=None, factor=2.5, clip_range=(0, 1), aoi_extent=aoi_coords_wgs84, title="Race_Publica: Sentinel-2 + Vector Data"):
    """
    Displays a raster image with overlaid vector data.
    Now can overlay waterways and land in different colors.
    """
    fig, ax = plt.subplots(figsize=(12, 12))
    extent = [aoi_extent[0], aoi_extent[2], aoi_extent[1], aoi_extent[3]]
    raster_display = np.clip(raster_image * factor, *clip_range)
    ax.imshow(raster_display, extent=extent, origin='upper')

    if gdf_waterways is not None and not gdf_waterways.empty:
        # Waterways: Yellow
        gdf_waterways.plot(ax=ax, color='yellow', linewidth=2.5, alpha=0.9, label='Waterways (OSM)')

    if gdf_land is not None and not gdf_land.empty:
        # Land (inverse): Lime Green
        gdf_land.plot(ax=ax, color='lime', edgecolor='green', linewidth=0.5, alpha=0.6, label='Landmass (Inverse)')

    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_xlim(aoi_extent[0], aoi_extent[2])
    ax.set_ylim(aoi_extent[1], aoi_extent[3])
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, "overlay_visual.png"))
    print(f"✅ Final overlay visualization saved.")
    plt.show()

def plot_geojson_only(vector_gdf, aoi_extent=aoi_coords_wgs84, title="GeoJSON Features"):
    """Displays only vector data."""
    fig, ax = plt.subplots(figsize=(10, 10))
    if not vector_gdf.empty:
        color = 'darkgreen' if 'Landmass' in title else 'blue'
        edgecolor = 'green' if 'Landmass' in title else 'darkblue'
        vector_gdf.plot(ax=ax, color=color, linewidth=2, edgecolor=edgecolor, alpha=0.7)

    ax.set_title(f"{title} - {len(vector_gdf)} objects", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(aoi_extent[0], aoi_extent[2])
    ax.set_ylim(aoi_extent[1], aoi_extent[3])
    plt.tight_layout()
    visual_path = os.path.join(OUTPUT_DIR, title.replace(" ", "_").lower().replace(":", "") + ".png")
    plt.savefig(visual_path)
    print(f"✅ GeoJSON visualization saved as: {visual_path}")
    plt.show()


# --- 3. CREDENTIALS SETUP ---

print("\n--- 3. Sentinel Hub Credentials Setup ---")
config = SHConfig()

# >>>>>>>>> EMBEDDED REAL DATA FOR DEBUGGING <<<<<<<<<
config.sh_client_id = "sh-2d66e1f7-ba1d-4bce-88bc-b7baffe1378f"
config.sh_client_secret = "UYNkQKoAe0Z3abO4dOTPZuv6FPF1gphd"
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

if config.sh_client_id and config.sh_client_secret:
    print("✅ Sentinel Hub configuration for CDSE successfully set.")
else:
    raise SystemExit("❌ CRITICAL ERROR: Credentials missing.")


# --- 4. TRUE COLOR (RGB) DOWNLOAD AND VISUALIZATION ---

print("\n--- 4. True Color (RGB) Download ---")
evalscript_float_color = """
    //VERSION=3
    function setup() {
        return {
            input: [{ bands: ["B02", "B03", "B04"], units: "REFLECTANCE" }],
            output: { bands: 3, sampleType: "FLOAT32" }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""
request_true_color = SentinelHubRequest(
    evalscript=evalscript_float_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(name="s2l2a", service_url=config.sh_base_url),
            time_interval=time_interval,
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox, size=aoi_size, config=config,
)
try:
    true_color_imgs = request_true_color.get_data()
    if not true_color_imgs:
        raise Exception("Failed to download RGB image.")
    image_rgb_loaded = true_color_imgs[0]
    print(f"✅ True Color Image obtained with shape: {image_rgb_loaded.shape}")

    save_raster_data(image_rgb_loaded, aoi_coords_wgs84, aoi_size, "s2_rgb_berlin.tif")

    plot_image(image_rgb_loaded, factor=2.5, clip_range=(0, 1), title="True Color Image (Scaled FLOAT32)")
    plt.savefig(os.path.join(OUTPUT_DIR, "s2_rgb_visual.png"))
    print(f"✅ Visualization saved as: {OUTPUT_DIR}/s2_rgb_visual.png")
except Exception as e:
    print(f"❌ RGB download error: {e}")
    image_rgb_loaded = np.zeros((aoi_size[1], aoi_size[0], 3), dtype=np.float32)
    print("❌ Using RGB placeholder.")


# --- 5. VECTOR DATA DOWNLOAD (OSM) ---

# AOI: [West, South, East, North]
W, S, E, N = aoi_coords_wgs84
# Create a polygon covering the entire AOI
polygon_geom_aoi = Polygon([
    (W, S), (W, N), (E, N), (E, S), (W, S)
])
geojsoп_waterways_filepath = os.path.join(OUTPUT_DIR, GEOJSON_WATERWAYS_FILENAME)


# --- 5.1. GeoJSON Waterways Download (NODES) ---
print("\n--- 5.1. GeoJSON Waterways Download (OSM) ---")
tags_lines = {'waterway': ['river', 'canal', 'stream', 'drain']}
tags_poly = {'natural': 'water'}
gdf_waterways = gpd.GeoDataFrame(geometry=[], crs=CRS.WGS84.ogc_string())

try:
    print("⏳ Downloading waterway data using ox.features_from_polygon...")

    gdf_lines = ox.features_from_polygon(polygon_geom_aoi, tags_lines)
    gdf_poly = ox.features_from_polygon(polygon_geom_aoi, tags_poly)

    gdf_lines = gdf_lines[gdf_lines.geometry.geom_type.isin(['LineString', 'MultiLineString'])][['geometry']]
    gdf_poly = gdf_poly[gdf_poly.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])][['geometry']]

    # Combine ALL water objects (lines and polygons)
    gdf_waterways = pd.concat([gdf_lines.geometry, gdf_poly.geometry], ignore_index=True)
    gdf_waterways = gpd.GeoDataFrame(geometry=gdf_waterways, crs=CRS.WGS84.ogc_string())

    # Convert lines to polygons for simplified inversion calculation (buffering)
    gdf_waterways['geometry'] = gdf_waterways.geometry.apply(lambda geom: geom.buffer(0.0001) if geom.geom_type in ['LineString', 'MultiLineString'] else geom)

    if len(gdf_waterways) > 0:
        gdf_waterways.to_file(geojson_waterways_filepath, driver='GeoJSON')
        print(f"✅ GeoJSON waterways saved. Objects: {len(gdf_waterways)}")
    else:
        print("⚠️ WARNING: 0 OSM objects downloaded for waterways.")

except Exception as e:
    print(f"❌ CRITICAL ERROR downloading GeoJSON waterways: {e}")


# 📢 UPDATED STEP: Creating land by inversion
# --- 5.2. Create GeoJSON Land (Inversion: AOI - Water) ---
print("\n--- 5.2. Create GeoJSON Land (Inversion) ---")

land_geojson_filepath = os.path.join(OUTPUT_DIR, LAND_GEOJSON_FILENAME)
gdf_land_final = gpd.GeoDataFrame(geometry=[], crs=CRS.WGS84.ogc_string())

if len(gdf_waterways) > 0:
    try:
        print("⏳ Calculating union of waterways (unary_union)...")
        # Combine all water geometries into a single MultiPolygon
        water_union = unary_union(gdf_waterways.geometry)

        print("⏳ Calculating inversion: AOI polygon minus Water (difference)...")
        # Inversion: Landmass = AOI polygon minus combined Water
        land_mass_geom = polygon_geom_aoi.difference(water_union)

        # Create GeoDataFrame for landmass
        gdf_land_final = gpd.GeoDataFrame(geometry=[land_mass_geom], crs=CRS.WGS84.ogc_string())

        if not gdf_land_final.empty:
            gdf_land_final.to_file(land_geojson_filepath, driver='GeoJSON')
            print(f"✅ GeoJSON landmass (inversion) saved: {land_geojson_filepath}")
            print(f"   Created {len(gdf_land_final)} objects (usually 1 MultiPolygon).")

            # Visualize landmass separately
            plot_geojson_only(gdf_land_final, aoi_extent=aoi_coords_wgs84, title="Berlin Landmass (Inverse AOI - Water)")

    except Exception as e:
        print(f"❌ Error calculating landmass inversion: {e}")
        gpd.GeoDataFrame(geometry=[], crs=CRS.WGS84.ogc_string()).to_file(land_geojson_filepath, driver='GeoJSON')
else:
    print("⚠️ No waterways for inversion. GeoJSON landmass will be the AOI polygon.")
    gdf_land_final = gpd.GeoDataFrame(geometry=[polygon_geom_aoi], crs=CRS.WGS84.ogc_string())
    gdf_land_final.to_file(land_geojson_filepath, driver='GeoJSON')


# --- 5.3. Vector Data Visualization on RGB ---
# (Using the updated plot_overlay function)
try:
    if os.path.exists(os.path.join(OUTPUT_DIR, "s2_rgb_berlin.tif")):
        with rasterio.open(os.path.join(OUTPUT_DIR, "s2_rgb_berlin.tif")) as src:
            image_loaded_for_overlay = np.transpose(src.read(), (1, 2, 0))

        print("\n--- 5.3. Waterways (Yellow) and Land (Green) Visualization on RGB ---")
        plot_overlay(image_loaded_for_overlay, gdf_waterways=gdf_waterways, gdf_land=gdf_land_final,
                     aoi_extent=aoi_coords_wgs84, title="Race_Publica: Waterways (Yellow) & Landmass (Lime) on RGB")
    else:
        print("❌ Could not find 's2_rgb_berlin.tif' for overlay visualization.")
except Exception as e:
    print(f"❌ Error loading or visualizing RGB for overlay: {e}")


# --- 6. NDWI BANDS DOWNLOAD (B03, B08) ---
# ... (NO CODE CHANGES) ...
print("\n--- 6. NDWI Bands Download (B03, B08) ---")
evalscript_ndwi_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{ bands: ["B03", "B08"], units: "REFLECTANCE" }],
            output: { bands: 2, sampleType: "FLOAT32" }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B03, sample.B08];
    }
"""
request_ndwi_bands = SentinelHubRequest(
    evalscript=evalscript_ndwi_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(name="s2l2a", service_url=config.sh_base_url),
            time_interval=time_interval,
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox, size=aoi_size, config=config,
)

try:
    ndwi_bands_imgs = request_ndwi_bands.get_data()
    if not ndwi_bands_imgs:
        raise Exception("Failed to download NDWI Bands.")
    ndwi_image_3d = ndwi_bands_imgs[0]
    transform = save_raster_data(ndwi_image_3d, aoi_coords_wgs84, aoi_size, "s2_ndwi_bands_berlin.tif")

except Exception as e:
    print(f"❌ NDWI Bands download error: {e}")
    ndwi_image_3d = np.zeros((aoi_size[1], aoi_size[0], 2), dtype=np.float32)
    lon_min, lat_min, lon_max, lat_max = aoi_coords_wgs84
    width, height = aoi_size
    transform = Affine(
        (lon_max - lon_min) / width, 0.0, lon_min,
        0.0, -(lat_max - lat_min) / height, lat_max
    )


# --- 7. NDWI SCORING CALCULATION AND SENSITIVITY ANALYSIS ---
# ... (NO CODE CHANGES) ...
print("\n--- 7. NDWI Scoring Calculation and Sensitivity Analysis ---")
B03 = ndwi_image_3d[:, :, 0].astype('float32')
B08 = ndwi_image_3d[:, :, 1].astype('float32')

with np.errstate(divide='ignore', invalid='ignore'):
    ndwi_map = (B03 - B08) / (B03 + B08)
ndwi_map[np.isnan(ndwi_map)] = 0
ndwi_map[np.isinf(ndwi_map)] = 0
ndwi_water_only = ndwi_map.copy()

if not gdf_waterways.empty:
    shapes_with_values = [(geom, 1) for geom in gdf_waterways.geometry]
    water_mask = rasterize(
        shapes=shapes_with_values,
        out_shape=ndwi_map.shape,
        transform=transform,
        fill=0,
        all_touched=True,
        dtype=np.uint8
    )
    ndwi_water_only = np.where(water_mask == 1, ndwi_map, np.nan)

# Scoring visualization
fig, ax = plt.subplots(figsize=(12, 12))
extent = [aoi_coords_wgs84[0], aoi_coords_wgs84[2], aoi_coords_wgs84[1], aoi_coords_wgs84[3]]
im = ax.imshow(
    ndwi_water_only, extent=extent, cmap='RdYlGn', vmin=0.1, vmax=0.6, origin='upper'
)
ax.set_title("Environmental Sensitivity Map (NDWI within GeoJSON)", fontsize=16)
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('NDWI Value (Environmental Scoring)', rotation=270, labelpad=15)
visual_path_sensitivity = os.path.join(OUTPUT_DIR, "sensitivity_map_ndwi.png")
plt.savefig(visual_path_sensitivity)
print(f"✅ Sensitivity map saved as: {visual_path_sensitivity}")
plt.show()


# --- 8. CREATE GEOJSON WITH NDWI PIXELS ---
# ... (NO CODE CHANGES) ...
print("\n--- 8. Create GeoJSON with NDWI Pixels ---")
geojsoп_ndwi_path = os.path.join(OUTPUT_DIR, "ndwi_pixels_water.geojson")

if not gdf_waterways.empty:
    valid_ndwi_mask = ~np.isnan(ndwi_water_only)
    ndwi_values = ndwi_water_only[valid_ndwi_mask]

    rows_indices, cols_indices = np.where(valid_ndwi_mask)

    if ndwi_values.size > 0:
        longitudes, latitudes = rasterio.transform.xy(transform, rows_indices, cols_indices, offset='center')

        ndwi_points_gdf = gpd.GeoDataFrame(
            data={'NDWI': ndwi_values},
            geometry=gpd.points_from_xy(longitudes, latitudes),
            crs=CRS.WGS84.ogc_string()
        )

        ndwi_points_gdf.to_file(geojson_ndwi_path, driver='GeoJSON')
        print(f"✅ GeoJSON with NDWI pixels saved: {geojson_ndwi_path}")
        print(f"   Number of points: {len(ndwi_points_gdf)}")
    else:
        print("❌ No valid NDWI values found on waterways for export.")
else:
    print("❌ No waterway vector data loaded. NDWI GeoJSON not created.")

In [ ]:
# -----------------------------------------------------
# File: 02_graph_builder_and_pathfinder.py
# Graph construction and interactive selection of 3 routes.
# NEW: Displaying computation time for each stage.
# CHANGED: Automatic configuration saving (with int64 fix).
# 💡 UPDATED: Preview path colors match 04_visualize_results.py (Red/Blue dashed)
# -----------------------------------------------------

import geopandas as gpd
from shapely.geometry import Point, LineString
from ipyleaflet import Map, basemaps, GeoJSON, Marker
from IPython.display import display, HTML
import networkx as nx
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os
import time
import pickle
import pandas as pd
import rasterio
from ipyleaflet import AwesomeIcon
import json

# 📢 ADDITIONAL IMPORTS FOR COLOR SCALE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors

# 📢 CONSTANTS, consistent with 00_data_processing.py
OUTPUT_DIR = "race_publica_data"
GEOJSON_NODES_FILE = os.path.join(OUTPUT_DIR, 'ndwi_pixels_water.geojson')
GEOJSON_LAND_POLYGON_FILE = os.path.join(OUTPUT_DIR, 'landmass_from_water_inverse.geojson')
GEOJSON_LAND_PIXELS_FILE = os.path.join(OUTPUT_DIR, 'landmass_pixels.geojson')

GRAPH_FILE = os.path.join(OUTPUT_DIR, 'optimized_water_graph.gpickle')
R = 0.0002              # Graph radius
CLICK_RADIUS = 0.0001 # Radius for detecting clicks on nodes
NDWI_LAND_VALUE = -1.0 # NDWI value for land

# <<< NEW CONFIGURATION FILE >>>
BOAT_CONFIGS_FILE = os.path.join(OUTPUT_DIR, 'boat_configs.json')

# 📢 COLOR CONSTANTS FOR OPTIMAL ROUTES (UNIFIED WITH 04_visualize_results.py)
COLOR_OPTIMAL_DIST_RED = '#FF0000' # Red (for weight_dist)
COLOR_OPTIMAL_NDWI_BLUE = '#0000FF' # Blue (for weight_ndwi)

# --- GLOBAL VARIABLES FOR STORING ROUTES ---
clicked_nodes = []
NUM_ROUTES = 3
# ------------------------------------------------------


# --- 0. ADDITIONAL STEP: Creating land pixels (if missing) ---
print("⏳ STAGE 0: Creating land pixels (points) from polygon...")
start_time_0 = time.time()

land_pixels_created = False

if not os.path.exists(GEOJSON_LAND_PIXELS_FILE) and os.path.exists(GEOJSON_LAND_POLYGON_FILE):
    try:
        # Load land polygon and water nodes to get transform
        gdf_land_polygon = gpd.read_file(GEOJSON_LAND_POLYGON_FILE)
        gdf_water_nodes = gpd.read_file(GEOJSON_NODES_FILE)

        # Get transform from GeoTIFF, if it exists (or from water GeoJSON)
        try:
            with rasterio.open(os.path.join(OUTPUT_DIR, "s2_ndwi_bands_berlin.tif")) as src:
                transform = src.transform
                out_shape = src.shape
        except Exception:
            # Approximate calculation of transform
            x_min = gdf_water_nodes.geometry.x.min()
            y_min = gdf_water_nodes.geometry.y.min()
            x_max = gdf_water_nodes.geometry.x.max()
            y_max = gdf_water_nodes.geometry.y.max()

            from rasterio.transform import from_bounds
            width = 330
            height = 127
            transform = from_bounds(x_min, y_min, x_max, y_max, width, height)
            out_shape = (height, width)


        # Rasterization of the land polygon
        shapes_with_values = [(geom, 1) for geom in gdf_land_polygon.geometry]
        land_mask = rasterio.features.rasterize(
            shapes=shapes_with_values,
            out_shape=out_shape,
            transform=transform,
            fill=0,
            all_touched=True,
            dtype=np.uint8
        )

        # Conversion of mask to points
        rows_indices, cols_indices = np.where(land_mask == 1)
        longitudes, latitudes = rasterio.transform.xy(transform, rows_indices, cols_indices, offset='center')

        gdf_land_pixels = gpd.GeoDataFrame(
            data={'NDWI': [NDWI_LAND_VALUE] * len(longitudes), 'type': ['land'] * len(longitudes)},
            geometry=gpd.points_from_xy(longitudes, latitudes),
            crs=gdf_water_nodes.crs
        )

        # Remove pixels that overlap with water nodes
        land_coords = set(gdf_land_pixels.geometry.apply(lambda p: (p.x, p.y)))
        water_coords = set(gdf_water_nodes.geometry.apply(lambda p: (p.x, p.y)))

        unique_land_coords = land_coords - water_coords

        gdf_land_pixels = gdf_land_pixels[gdf_land_pixels.geometry.apply(lambda p: (p.x, p.y) in unique_land_coords)].reset_index(drop=True)


        gdf_land_pixels.to_file(GEOJSON_LAND_PIXELS_FILE, driver='GeoJSON')
        print(f"    ✅ Created {len(gdf_land_pixels)} land pixels: {GEOJSON_LAND_PIXELS_FILE}")
        land_pixels_created = True

    except Exception as e:
        print(f"    ❌ Error during land pixel generation: {e}")

end_time_0 = time.time()
print(f"⏱️ STAGE 0 completed in: {end_time_0 - start_time_0:.2f} sec.")


# --- 1. Loading and preparing data ---
print("\n⏳ STAGE 1: Loading GeoJSON nodes (water and land)...")
start_time_1 = time.time()

try:
    # 1.1. Loading WATER GeoJSON nodes
    if not os.path.exists(GEOJSON_NODES_FILE):
        raise FileNotFoundError(f"Water nodes file not found: {GEOJSON_NODES_FILE}")

    gdf_water_nodes = gpd.read_file(GEOJSON_NODES_FILE)
    if 'type' not in gdf_water_nodes.columns:
        gdf_water_nodes['type'] = 'water'
    print(f"    ✅ Water nodes loaded: {len(gdf_water_nodes)}.")

    # 1.2. Loading LAND GeoJSON nodes
    if os.path.exists(GEOJSON_LAND_PIXELS_FILE):
        gdf_land_nodes = gpd.read_file(GEOJSON_LAND_PIXELS_FILE)
        if 'type' not in gdf_land_nodes.columns:
            gdf_land_nodes['type'] = 'land'

        # Add NDWI value for land, if missing
        if 'NDWI' not in gdf_land_nodes.columns:
            gdf_land_nodes['NDWI'] = NDWI_LAND_VALUE

        print(f"    ✅ Land nodes loaded: {len(gdf_land_nodes)}.")
    else:
        gdf_land_nodes = gpd.GeoDataFrame(geometry=[], crs=gdf_water_nodes.crs)
        print(f"    ❌ Land nodes file '{GEOJSON_LAND_PIXELS_FILE}' not found. Land will not be considered.")


    # 1.3. Combining all nodes
    if gdf_water_nodes.empty:
        raise Exception("No water nodes for graph construction.")

    # Align columns before concatenation
    required_cols = ['NDWI', 'geometry', 'type']
    for gdf in [gdf_water_nodes, gdf_land_nodes]:
        for col in required_cols:
            if col not in gdf.columns:
                if col == 'NDWI' and 'type' in gdf.columns and not gdf.empty and gdf['type'].iloc[0] == 'land':
                    gdf[col] = NDWI_LAND_VALUE
                else:
                    gdf[col] = None
        gdf = gdf[required_cols]


    gdf_all_nodes = pd.concat([gdf_water_nodes, gdf_land_nodes], ignore_index=True)
    gdf_all_nodes['node_id'] = range(len(gdf_all_nodes))

    print(f"    ➡️ All points for the graph: {len(gdf_all_nodes)}.")

except FileNotFoundError as e:
    print(f"❌ CRITICAL ERROR: {e}. Please run '00_data_processing.py' first.")
    # Create minimal test data if all else fails
    data = {'NDWI': [0.5], 'geometry': [Point(13.25, 52.57)], 'type': ['water']}
    gdf_all_nodes = gpd.GeoDataFrame(data, crs="EPSG:4326")
    gdf_all_nodes['node_id'] = range(len(gdf_all_nodes))
except Exception as e:
    print(f"❌ CRITICAL ERROR: {e}")
    data = {'NDWI': [0.5], 'geometry': [Point(13.25, 52.57)], 'type': ['water']}
    gdf_all_nodes = gpd.GeoDataFrame(data, crs="EPSG:4326")
    gdf_all_nodes['node_id'] = range(len(gdf_all_nodes))


end_time_1 = time.time()
print(f"⏱️ STAGE 1 completed in: {end_time_1 - start_time_1:.2f} sec.")

# Calculate map center
center_lat = gdf_all_nodes.geometry.y.mean() if not gdf_all_nodes.empty else 52.57
center_lon = gdf_all_nodes.geometry.x.mean() if not gdf_all_nodes.empty else 13.25


# --- 2. Creating / Loading OPTIMIZED Graph (CACHING) ---
print("\n⏳ STAGE 2: Building/loading graph with land check...")
start_time_2 = time.time()

G = None
should_build_new_graph = True

# Cache check logic
if os.path.exists(GRAPH_FILE):
    try:
        with open(GRAPH_FILE, 'rb') as f:
            G_cached = pickle.load(f)

        # Check: R and number of nodes
        if (
            G_cached.graph.get('R_value') == R and
            G_cached.number_of_nodes() == len(gdf_all_nodes)
        ):

            G = G_cached
            print(f"    ✅ Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges (R={R}).")
            should_build_new_graph = False
        else:
            print(f"    ❌ R parameters or number of nodes changed. A new graph will be created.")
            should_build_new_graph = True
    except Exception as e:
        print(f"    ❌ Error loading graph: {e}. A new graph will be created.")
        should_build_new_graph = True

if should_build_new_graph:
    # >>> GRAPH CREATION LOGIC <<<

    if gdf_all_nodes.empty:
        print("❌ No nodes to build graph. Skipping STAGE 2.")
        G = nx.Graph(R_value=R)
    else:
        coords = np.array(list(zip(gdf_all_nodes.geometry.x, gdf_all_nodes.geometry.y)))

        nn = NearestNeighbors(radius=R, algorithm='ball_tree').fit(coords)
        distances_list, indices_list = nn.radius_neighbors(coords)

        G = nx.Graph(R_value=R)

        # NDWI normalization (for water nodes only)
        water_ndwi = gdf_all_nodes[gdf_all_nodes['type'] == 'water']['NDWI']
        min_ndwi_true = water_ndwi.min() if not water_ndwi.empty else 0.0

        gdf_all_nodes['NDWI_normalized'] = gdf_all_nodes['NDWI'] - min_ndwi_true
        epsilon = 0.001

        print(f"    ⚙️ Building graph with {len(gdf_all_nodes)} points...")

        for i in range(len(gdf_all_nodes)):
            row_i = gdf_all_nodes.iloc[i]
            point_i = row_i.geometry

            G.add_node(row_i['node_id'], x=point_i.x, y=point_i.y, ndwi=row_i['NDWI'], type=row_i['type'])

            ndwi_cost_i = 1.0 / (row_i['NDWI_normalized'] + epsilon) if row_i['type'] == 'water' else float('inf')

            neighbors_indices = indices_list[i]
            neighbors_distances = distances_list[i]

            for j in range(len(neighbors_indices)):
                target_index = neighbors_indices[j]
                distance = neighbors_distances[j]

                if i >= target_index:
                    continue

                row_j = gdf_all_nodes.iloc[target_index]

                # LOGIC: Create edge ONLY between two water nodes
                if row_i['type'] == 'water' and row_j['type'] == 'water':
                    ndwi_cost_j = 1.0 / (row_j['NDWI_normalized'] + epsilon)

                    weight_dist = distance
                    avg_ndwi_cost = (ndwi_cost_i + ndwi_cost_j) / 2.0
                    weight_ndwi = distance * avg_ndwi_cost

                    G.add_edge(
                        row_i['node_id'],
                        row_j['node_id'],
                        weight_dist=weight_dist,
                        weight_ndwi=weight_ndwi
                    )

        print(f"    ✅ Graph created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

        # Save graph
        try:
            with open(GRAPH_FILE, 'wb') as f:
                pickle.dump(G, f)
            print(f"    💾 Graph saved as '{GRAPH_FILE}'.")
        except Exception as e:
            print(f"    ❌ Error saving graph: {e}")

end_time_2 = time.time()
print(f"⏱️ STAGE 2 completed in: {end_time_2 - start_time_2:.2f} sec.")

if G is None:
    G = nx.Graph(R_value=R)


# --- 3. FUNCTIONS FOR INTERACTIVE ROUTE SELECTION ---

def find_nearest_node(lat, lon):
    """Finds the closest WATER NODE to the click within CLICK_RADIUS."""
    gdf_water_for_nn = gdf_all_nodes[gdf_all_nodes['type'] == 'water']
    if gdf_water_for_nn.empty: return None
    coords_for_nn = np.array(list(zip(gdf_water_for_nn.geometry.y, gdf_water_for_nn.geometry.x)))
    nn_search = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(coords_for_nn)
    distances, indices = nn_search.kneighbors([[lat, lon]])
    nearest_distance = distances[0][0]
    if nearest_distance > CLICK_RADIUS: return None
    nearest_index_in_water_gdf = indices[0][0]
    node_id = gdf_water_for_nn.iloc[nearest_index_in_water_gdf]['node_id']
    return node_id

def create_path_geojson(path_nodes, start_point, end_point, color, dash_array=''):
    """Creates a GeoJSON object for the path (Used only for preview)."""
    path_geometries = [
        gdf_all_nodes[gdf_all_nodes['node_id'] == node_id].geometry.iloc[0]
        for node_id in path_nodes
    ]
    # In this version of the code, the endpoint is already added to path_geometries if the path exists
    final_path_coords = [gdf_all_nodes[gdf_all_nodes['node_id'] == path_nodes[0]].geometry.iloc[0]] + path_geometries[1:]
    path_line = LineString([(p.x, p.y) for p in final_path_coords])
    gdf_path = gpd.GeoDataFrame([1], geometry=[path_line], crs=gdf_all_nodes.crs)

    style = {'color': color, 'weight': 5, 'opacity': 0.8}
    if dash_array:
        style['dashArray'] = dash_array
        style['weight'] = 2 # Reduce thickness for dashed lines

    return GeoJSON(
        data=gdf_path.__geo_interface__,
        style=style
    )

def clear_path_layers():
    """Removes all path layers and click markers from the map."""
    global m
    # Remove path layers and click markers
    # Path colors: #FF0000 (Red), #0000FF (Blue)
    path_colors = [COLOR_OPTIMAL_DIST_RED, COLOR_OPTIMAL_NDWI_BLUE]
    layers_to_remove = [layer for layer in list(m.layers) if getattr(layer, 'style', {}).get('color') in path_colors or isinstance(layer, Marker) and not getattr(layer, 'title', '').startswith('NDWI')]
    for layer in layers_to_remove:
        m.remove_layer(layer)


def generate_simulation_config_and_save(nodes_list):
    """
    Generates boat configuration, converting numpy.int64 to int,
    and AUTOMATICALLY saves it to a JSON file.
    """
    if len(nodes_list) != NUM_ROUTES * 2:
        return

    # !!! FIX: Convert numpy.int64 to standard Python int
    nodes = [int(n[0]) for n in nodes_list]

    boat_configs = [
        # --- ROUTE 1: Shortest Path (Boat 1) ---
        {
            'id': 'Boat_Shortest',
            'start_node': nodes[0],
            'end_node': nodes[1],
            'strategy': 'shortest_dist', # Chooses the shortest path
            'prob': 0.05,
            'radius': 1
        },
        # --- ROUTE 2: Greenest Path (Boat 2) ---
        {
            'id': 'Boat_Greenest',
            'start_node': nodes[2],
            'end_node': nodes[3],
            'strategy': 'best_ndwi', # Chooses the greenest path
            'prob': 0.15,
            'radius': 1
        },
        # --- ROUTE 3: Chaotic Path (Boat 3) ---
        {
            'id': 'Boat_Chaotic',
            'start_node': nodes[4],
            'end_node': nodes[5],
            'strategy': 'chaotic', # Random path selection (50/50)
            'prob': 0.3,
            'radius': 3
        },
    ]

    try:
        # Check and create directory if it doesn't exist
        if not os.path.exists(OUTPUT_DIR):
             os.makedirs(OUTPUT_DIR)

        with open(BOAT_CONFIGS_FILE, 'w') as f:
            json.dump(boat_configs, f, indent=4)

        print(f"\n✅ CONFIGURATION AUTOMATICALLY SAVED. ")
        print(f"💾 Route configuration saved to file: **{BOAT_CONFIGS_FILE}**")
        print("\n👉 Now you can run `03_boat_simulator_and_scorer.py`!")

    except Exception as e:
        print(f"\n❌ CRITICAL ERROR DURING AUTO-SAVING CONFIGURATION: {e}")
        # Fallback output of code in case of saving error
        print("Outputting configuration for manual copying as a fallback option:")
        code_string = f"# ==============================================================================\n# 💡 BOAT_CONFIGS (FALLBACK)\n# ==============================================================================\nBOAT_CONFIGS = {json.dumps(boat_configs, indent=4)}\n\nNUM_BOATS = len(BOAT_CONFIGS)"
        display(HTML(f'<pre style="background-color: #f4f4f4; border: 1px solid #ddd; padding: 10px; overflow-x: auto;">{code_string.replace("<", "&lt;").replace(">", "&gt;")}</pre>'))


def handle_click(**kwargs):
    """Function called on map click to save nodes."""
    global m, clicked_nodes

    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')

        # 1. Find the nearest water node
        node_id = find_nearest_node(lat, lon)

        if node_id is None:
            print(f"\n❌ Error: Click made too far from water points (radius {CLICK_RADIUS}). Try clicking more precisely.")
            return

        # 2. Save node and coordinates
        if len(clicked_nodes) < NUM_ROUTES * 2:
            clicked_nodes.append((node_id, lat, lon))
        else:
             # This is needed if someone clicks after completion
             print(f"\n📢 All {NUM_ROUTES} routes already selected. Clear the list or restart for new selection.")
             return


        idx = len(clicked_nodes)
        route_num = (idx - 1) // 2 + 1
        point_type = "START 🚀" if idx % 2 != 0 else "FINISH 🏁"

        # 3. Add marker
        marker_color = 'blue' if idx % 2 != 0 else 'red'
        marker_icon = AwesomeIcon(name='location-dot', marker_color=marker_color, icon_color='white', prefix='fa')
        marker_title = f"Route {route_num}: {point_type} (Node: {node_id})"

        new_marker = Marker(location=(lat, lon), draggable=False, icon=marker_icon, title=marker_title)

        if idx == 1:
             clear_path_layers() # Clear old paths and markers at the start of a new cycle

        m.add_layer(new_marker)

        print(f"\nClick {idx} saved: {point_type} Route {route_num} (Node ID: {node_id}).")


        if len(clicked_nodes) == NUM_ROUTES * 2:
            print("\n==================================================================================")
            print(f"✅ All {NUM_ROUTES} routes selected! Starting STAGE 3 (Path preview)...")
            print("==================================================================================")

            # --- STAGE 3: Route preview ---
            start_time_3 = time.time()
            # clear_path_layers() # Remove all previous paths

            total_red_time = 0
            total_green_time = 0

            for i in range(NUM_ROUTES):
                start_node_id, start_lat, start_lon = clicked_nodes[i * 2]
                end_node_id, end_lat, end_lon = clicked_nodes[i * 2 + 1]

                # For correct path display, use graph points:
                start_point = gdf_all_nodes[gdf_all_nodes['node_id'] == start_node_id].geometry.iloc[0]
                end_point = gdf_all_nodes[gdf_all_nodes['node_id'] == end_node_id].geometry.iloc[0]


                # Search and display RED (Distance)
                start_time_red = time.time()
                try:
                    path_dist_nodes = nx.shortest_path(G, source=start_node_id, target=end_node_id, weight='weight_dist')
                    # 💡 UPDATED: RED and DASHED
                    path_dist_geojson = create_path_geojson(path_dist_nodes, start_point, end_point, COLOR_OPTIMAL_DIST_RED, dash_array='4, 4')
                    m.add_layer(path_dist_geojson)
                    time_red = time.time() - start_time_red
                    total_red_time += time_red
                    print(f"    1. ✅ RED (Distance) Route {i+1} found. Time: {time_red:.4f} sec.")
                except nx.NetworkXNoPath:
                    print(f"    ❌ Distance path for route {i+1} not found.")

                # Search and display BLUE (NDWI)
                start_time_green = time.time()
                try:
                    path_ndwi_nodes = nx.shortest_path(G, source=start_node_id, target=end_node_id, weight='weight_ndwi')
                    # 💡 UPDATED: BLUE and DASHED
                    path_ndwi_geojson = create_path_geojson(path_ndwi_nodes, start_point, end_point, COLOR_OPTIMAL_NDWI_BLUE, dash_array='4, 4')
                    m.add_layer(path_ndwi_geojson)
                    time_green = time.time() - start_time_green
                    total_green_time += time_green
                    print(f"    2. ✅ BLUE (NDWI) Route {i+1} found. Time: {time_green:.4f} sec.")
                except nx.NetworkXNoPath:
                    print(f"    ❌ NDWI path for route {i+1} not found.")

            end_time_3 = time.time()
            print(f"⏱️ STAGE 3 completed in: {end_time_3 - start_time_3:.2f} sec. (Total search time: {total_red_time + total_green_time:.4f} sec.)")

            # --- AUTOMATIC CONFIGURATION SAVING (WITHOUT CODE OUTPUT) ---
            generate_simulation_config_and_save(clicked_nodes)

            # Clear clicks and prepare for a new run
            clicked_nodes.clear()


# --- 5. Displaying Map (NDWI-scoring) ---

print("\n⏳ STAGE 4: Displaying map (NDWI-scoring)...")
start_time_4 = time.time()

# 1. Create map object
m = Map(
    center=(center_lat, center_lon),
    zoom=14,
    tiles=basemaps.CartoDB.DarkMatter
)

if not gdf_all_nodes.empty:

    # 2. Filter water nodes
    gdf_water_for_viz = gdf_all_nodes[gdf_all_nodes['type'] == 'water'].copy()

    if not gdf_water_for_viz.empty:
        # 3. Define NDWI range for color scale
        min_ndwi = gdf_water_for_viz['NDWI'].min()
        max_ndwi = gdf_water_for_viz['NDWI'].max()
        ndwi_range = max_ndwi - min_ndwi

        # 4. Select color scale (RdYlGn - Red-Yellow-Green)
        cmap = plt.colormaps['RdYlGn']

        # 5. Assign color to each node based on NDWI
        def get_color(ndwi_value):
            if ndwi_range == 0:
                norm_ndwi = 0.5
            else:
                # Normalize from 0 to 1
                norm_ndwi = np.clip((ndwi_value - min_ndwi) / ndwi_range, 0, 1)

            # Get color in RGBA format
            rgba = cmap(norm_ndwi)
            # Convert to HEX string supported by GeoJSON (ipyleaflet)
            return matplotlib.colors.rgb2hex(rgba[:3])

        gdf_water_for_viz['color'] = gdf_water_for_viz['NDWI'].apply(get_color)

        # 6. Create GeoJSON for water points with dynamic color
        water_points_geojson = GeoJSON(
            data=gdf_water_for_viz.__geo_interface__,
            point_style={
                'radius': 3,
                'fillOpacity': 0.9,
                'weight': 0.5
            },
            style_callback=lambda feat: {
                'color': feat['properties']['color'],
                'fillColor': feat['properties']['color'],
            },
             name='NDWI Water Nodes'
        )
        m.add_layer(water_points_geojson)
        print("    ✅ Water nodes colored according to NDWI (Red=Low, Green=High).")

    # Add LAND (nodes) GeoDataFrame to the map (gray color)
    gdf_land_for_viz = gdf_all_nodes[gdf_all_nodes['type'] == 'land']
    if not gdf_land_for_viz.empty:
        land_points_geojson = GeoJSON(
            data=gdf_land_for_viz.__geo_interface__,
            style={'color': '#808080', 'fillColor': '#808080', 'weight': 1, 'radius': 2, 'opacity': 0.7, 'fillOpacity': 0.5},
            point_style={'radius': 2, 'color': '#808080', 'fillColor': '#808080', 'fillOpacity': 0.9, 'weight': 1},
             name='Land Nodes'
        )
        m.add_layer(land_points_geojson)


# Bind click handling function to the map
m.on_interaction(handle_click)

end_time_4 = time.time()
print(f"⏱️ STAGE 4 completed in: {end_time_4 - start_time_4:.2f} sec.")

print("\n✅ Ready. Instructions:")
print(" - **Step 1**: Click on the map 6 times (3 pairs: Start/Finish) for 3 routes.")
print(" - **Step 2**: After the 6th click, the configuration will be automatically saved to **race_publica_data/boat_configs.json**.")
print(f" - **R (Graph Radius)**: {R}")

# Add layer control for convenience
from ipyleaflet import LayersControl
m.add_control(LayersControl(position='topright'))

display(m)

In [ ]:
# ----------------------------------------------------------------------------------
# File: 03_boat_simulator_and_scorer.py
# SIMULATION STAGE: Loads configuration, runs boat simulation on the graph,
# and calculates final scoring.
# ----------------------------------------------------------------------------------

import numpy as np
import networkx as nx
import os
import time
import pickle
import json
import random
import math
import pandas as pd
from IPython.display import display

# 📢 CONSTANTS, consistent with 02_graph_builder_and_pathfinder.py
OUTPUT_DIR = "race_publica_data"
# FIX APPLIED HERE: Revert to using os.path.join with OUTPUT_DIR
GRAPH_FILE = os.path.join(OUTPUT_DIR, 'optimized_water_graph.gpickle')
BOAT_CONFIGS_FILE = os.path.join(OUTPUT_DIR, 'boat_configs.json')
SCORE_FILE = os.path.join(OUTPUT_DIR, 'simulation_scores.json')
SIMULATION_LOG_FILE = os.path.join(OUTPUT_DIR, 'simulation_log.json')

# --- SIMULATOR CONSTANTS ---
SIMULATION_STEPS = 1000
# 🚀 Increased base speed to ensure movement
BASE_SPEED = 0.0005 # Approximate distance per step (in degrees of latitude/longitude)
# 🌊 Noise factor for all boats
NOISE_FACTOR = 0.05 # Deviation factor from the ideal course (e.g., 0.05 = up to 5% angular deviation)

# --- Scoring settings ---
NDWI_SCORE_FACTOR = 1000  # Higher NDWI, better score
DISTANCE_SCORE_FACTOR = -10 # Longer path, worse score
TIME_PENALTY_FACTOR = 5   # Time penalty (depends on the number of steps)


# ==================================================================================
# --- JSON ERROR FIXING FUNCTION ---
# ==================================================================================

def convert_to_json_compatible(obj):
    """
    Recursively converts numpy objects to standard Python types.
    """
    if isinstance(obj, (np.int64, np.intc, np.intp, np.int8, np.int16, np.int32, np.uint8, np.uint16, np.uint32, np.uint64)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float16, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_to_json_compatible(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_compatible(elem) for elem in obj]
    else:
        return obj

# ==================================================================================
# --- MAIN SIMULATOR LOGIC ---
# ==================================================================================

def calculate_path(G, config):
    """Calculates the optimal path based on strategy."""
    start = config['start_node']
    end = config['end_node']
    strategy = config['strategy']

    # Using a fixed path (for "real" routes)
    if strategy == 'fixed_path' and config.get('path_nodes') is not None:
        print(f"  [Boat {config['id']}] Using a fixed path ('fixed_path').")
        return config['path_nodes']

    elif strategy == 'shortest_dist':
        weight = 'weight_dist'
    elif strategy == 'best_ndwi':
        weight = 'weight_ndwi'

    elif strategy == 'chaotic':
        # 💡 FIXED: Chaotic boat should only start from the start node,
        # as its path is dynamically built in move_boat.
        print(f"  [Boat {config['id']}] Not pre-calculating path (strategy: chaotic).")
        return [start] # RETURN ONLY THE START NODE

    else:
        weight = 'weight_dist'

    try:
        path = nx.shortest_path(G, source=start, target=end, weight=weight)
        return path
    except nx.NetworkXNoPath:
        print(f"  [Boat {config['id']}] Path from {start} to {end} not found.")
        return []

def move_boat(G, current_node_index, path_nodes, config, log_entry):
    """Simulates boat movement to the next node."""

    current_node_id = path_nodes[current_node_index]
    end_node_id = config['end_node'] # End node

    # 1. Check for finish
    if current_node_id == end_node_id:
        log_entry['finished'] = True
        return current_node_index, log_entry

    # 2. Determine the next node
    if config['strategy'] == 'chaotic':

        # 💡 CHAOTIC SELECTION LOGIC (dynamic)
        neighbors = list(G.neighbors(current_node_id))
        if not neighbors:
            log_entry['finished'] = True # Boat is stuck
            return current_node_index, log_entry

        # Calculate shortest path to end node to have a 'goal'
        try:
            shortest_path_to_end = nx.shortest_path(G, current_node_id, end_node_id, weight='weight_dist')
            optimal_neighbor = shortest_path_to_end[1] if len(shortest_path_to_end) > 1 else end_node_id
        except nx.NetworkXNoPath:
            optimal_neighbor = None


        if random.random() < config['prob']:
            # CHAOTIC CHOICE: select a random neighbor
            next_node_id = random.choice(neighbors)
            log_entry['chaotic_step'] = log_entry.get('chaotic_step', 0) + 1
        elif optimal_neighbor is not None and optimal_neighbor in neighbors:
            # NON-CHAOTIC CHOICE: select the optimal neighbor
            next_node_id = optimal_neighbor
        else:
            # Emergency mode
            next_node_id = random.choice(neighbors)

        # 3. Update path_nodes for dynamic path
        # If current index is the last in the path, add a new node.
        if len(path_nodes) == current_node_index + 1:
            path_nodes.append(next_node_id)

        # Move to the new node
        next_node_index = current_node_index + 1

    else:
        # LOGIC FOR SHORTEST/GREENEST (following a pre-calculated path)
        if current_node_index >= len(path_nodes) - 1:
            log_entry['finished'] = True
            return current_node_index, log_entry

        next_node_index = current_node_index + 1
        next_node_id = path_nodes[next_node_index]

    # --- Common movement and noise logic ---

    current_pos = np.array([G.nodes[current_node_id]['x'], G.nodes[current_node_id]['y']])
    next_pos = np.array([G.nodes[next_node_id]['x'], G.nodes[next_node_id]['y']])

    direction_vector = next_pos - current_pos
    distance = np.linalg.norm(direction_vector)

    # 🌊 ADDING NOISE: Random deviation from course
    if distance > 1e-6:
        angle_to_target = math.atan2(direction_vector[1], direction_vector[0])
        noise_angle = random.uniform(-math.pi * NOISE_FACTOR, math.pi * NOISE_FACTOR)
        new_angle = angle_to_target + noise_angle
        noisy_direction_vector = np.array([math.cos(new_angle), math.sin(new_angle)])
        direction_vector = noisy_direction_vector * distance

    # Influence of NDWI on speed
    ndwi_current = G.nodes[current_node_id].get('ndwi', 0.0)
    normalized_ndwi = (ndwi_current + 1.0) / 2.0
    speed_multiplier = 0.5 + normalized_ndwi

    # Measuring steps
    steps_needed = distance / BASE_SPEED

    if steps_needed * (1.0 / speed_multiplier) <= 1.0:
        current_node_index = next_node_index
        log_entry['current_node'] = path_nodes[current_node_index]
        log_entry['total_distance'] += distance
        log_entry['total_ndwi_sum'] += ndwi_current
        log_entry['ndwi_count'] += 1
    else:
        log_entry['current_node'] = path_nodes[current_node_index]

    return current_node_index, log_entry


def run_boat_simulation_and_score():
    """Main function to perform simulation and scoring."""
    print("\n==================================================================================")
    print("🌊 LAUNCHING SIMULATOR **03_boat_simulator_and_scorer.py**")
    print("==================================================================================")
    start_time_sim = time.time()

    # 1. Load Graph
    try:
        with open(GRAPH_FILE, 'rb') as f:
            G = pickle.load(f)
        print(f"✅ Graph loaded: {G.number_of_nodes()} nodes.")
    except FileNotFoundError:
        print(f"❌ Error: Graph file not found: {GRAPH_FILE}. Please run 02_graph_builder_and_pathfinder.py first.")
        return
    except Exception as e:
        print(f"❌ Error loading graph: {e}")
        return

    # 2. Load boat configurations
    try:
        with open(BOAT_CONFIGS_FILE, 'r') as f:
            boat_configs = json.load(f)
        NUM_BOATS = len(boat_configs)
        print(f"✅ Loaded configuration for {NUM_BOATS} boats.")
    except FileNotFoundError:
        print(f"❌ Error: Configuration file not found: {BOAT_CONFIGS_FILE}. Please run 02_graph_builder_and_pathfinder.py first.")
        return
    except json.JSONDecodeError:
        print("❌ Error: Incorrect JSON format in configuration file.")
        return

    # 3. Initialize simulation (calculate paths and update configuration)

    simulation_logs = []
    boat_states = {}
    updated_boat_configs = [] # To save calculated paths back to file

    for config in boat_configs:
        path = calculate_path(G, config)

        # 💡 Saving the calculated path in the configuration for visualization
        config['path_nodes'] = path
        updated_boat_configs.append(config)

        if not path:
            print(f" ❌ Warning: No path found for boat {config['id']}.")
            continue

        boat_states[config['id']] = {
            'path': path,
            'current_index': 0,
            'log': {
                'id': config['id'],
                'start_node': config['start_node'],
                'end_node': config['end_node'],
                'strategy': config['strategy'],
                'current_node': config['start_node'],
                'steps': 0,
                'total_distance': 0.0,
                'total_ndwi_sum': 0.0,
                'ndwi_count': 0,
                'finished': False
            }
        }

    # Replace initial configurations with updated ones
    boat_configs = updated_boat_configs

    # 4. Main simulation loop

    print("\n⚙️ Starting step-by-step simulation...")

    active_boats = list(boat_states.keys())

    for step in range(SIMULATION_STEPS):

        if not active_boats:
            print(f"  🎉 All boats finished at step {step}.")
            break

        new_active_boats = []

        for boat_id in active_boats:
            state = boat_states[boat_id]
            log = state['log']
            config = next((c for c in boat_configs if c['id'] == boat_id), None)

            if log['finished']:
                continue

            # --- Movement ---
            new_index, new_log = move_boat(G, state['current_index'], state['path'], config, log)

            state['current_index'] = new_index
            state['log'] = new_log
            state['log']['steps'] += 1

            if not state['log']['finished']:
                new_active_boats.append(boat_id)

        active_boats = new_active_boats

    # Collect final logs
    for boat_id in boat_states:
        simulation_logs.append(boat_states[boat_id]['log'])

    print(f"✅ Simulation completed (steps: {SIMULATION_STEPS}).")

    # 5. Save log
    try:
        json_log_compatible = convert_to_json_compatible(simulation_logs)
        if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
        with open(SIMULATION_LOG_FILE, 'w') as f:
            json.dump(json_log_compatible, f, indent=4)
        print(f"💾 Simulation log saved: **{SIMULATION_LOG_FILE}**")
    except Exception as e:
        print(f"❌ Error saving log: {e}")

    # 6. Scoring

    final_scores = []
    print("\n📊 Calculating final scoring...")

    for log in simulation_logs:
        total_distance = log['total_distance']
        total_ndwi_sum = log['total_ndwi_sum']
        ndwi_count = log['ndwi_count']
        steps = log['steps']

        avg_ndwi = total_ndwi_sum / ndwi_count if ndwi_count > 0 else 0.0

        time_penalty = steps * TIME_PENALTY_FACTOR

        ndwi_score = avg_ndwi * NDWI_SCORE_FACTOR
        distance_score = total_distance * DISTANCE_SCORE_FACTOR

        final_score = ndwi_score + distance_score - time_penalty

        final_scores.append({
            'id': log['id'],
            'strategy': log['strategy'],
            'score': round(final_score, 2),
            'avg_ndwi': round(avg_ndwi, 5),
            'distance': round(total_distance, 5),
            'steps': steps,
            'finished': log['finished']
        })

    # 7. Save scoring and configurations
    try:
        json_score_compatible = convert_to_json_compatible(final_scores)
        if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
        with open(SCORE_FILE, 'w') as f:
            json.dump(json_score_compatible, f, indent=4)
        print(f"✅ FINAL SCORING SAVED: **{SCORE_FILE}**")

        # 💡 Saving updated configuration with calculated paths
        json_configs_compatible = convert_to_json_compatible(boat_configs)
        with open(BOAT_CONFIGS_FILE, 'w') as f:
            json.dump(json_configs_compatible, f, indent=4)
        print(f"✅ BOAT CONFIGURATIONS UPDATED: **{BOAT_CONFIGS_FILE}**")

        scores_df = pd.DataFrame(final_scores).sort_values(by='score', ascending=False).reset_index(drop=True)
        scores_df['rank'] = scores_df.index + 1

        print("\n🏆 SIMULATION RESULTS (TOP-3):")
        # Use display for Jupyter/IPython
        display(scores_df[['rank', 'id', 'strategy', 'score', 'avg_ndwi', 'steps']])

    except Exception as e:
        print(f"❌ Error saving scoring: {e}")

    end_time_sim = time.time()
    print(f"\n⏱️ SIMULATOR completed in: {end_time_sim - start_time_sim:.2f} sec.")
    print("==================================================================================")


if __name__ == '__main__':
    run_boat_simulation_and_score()

In [ ]:
# ----------------------------------------------------------------------------------
# File: 04_visualize_results.py (CLEANED AND FIXED FOR 9 PATHS)
# VISUALIZATION STAGE: Loads data and displays ALL 9 paths:
# - 3 simulated (actual, bold lines)
# - 6 optimal (2 each: Dist/NDWI for every boat, dashed lines).
# ----------------------------------------------------------------------------------

import geopandas as gpd
from shapely.geometry import Point, LineString
from ipyleaflet import Map, basemaps, GeoJSON, Marker, AwesomeIcon
from IPython.display import display, Markdown
import networkx as nx
import numpy as np
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
import pickle
import rasterio

# 📢 CONSTANTS, consistent with previous stages
OUTPUT_DIR = "race_publica_data"
GEOJSON_NODES_FILE = os.path.join(OUTPUT_DIR, 'ndwi_pixels_water.geojson')
GEOJSON_LAND_PIXELS_FILE = os.path.join(OUTPUT_DIR, 'landmass_pixels.geojson')
GRAPH_FILE = os.path.join(OUTPUT_DIR, 'optimized_water_graph.gpickle')
BOAT_CONFIGS_FILE = os.path.join(OUTPUT_DIR, 'boat_configs.json')
SCORE_FILE = os.path.join(OUTPUT_DIR, 'simulation_scores.json') # File for final table
NDWI_LAND_VALUE = -1.0 # NDWI value for landmass

# --- COLORS FOR VISUALIZATION ---
# Simulated boat paths (Bold lines and icons)
COLOR_BOAT_1_ORANGE = '#FFA500' # Orange (Shortest)
COLOR_BOAT_2_PURPLE = '#800080' # Purple (Greenest)
COLOR_BOAT_3_PINK = '#FFC0CB' # Pink (Chaotic)

# Optimal paths (Thin dashed lines)
COLOR_OPTIMAL_DIST_RED = '#FF0000' # Red (Optimal Distance)
COLOR_OPTIMAL_NDWI_BLUE = '#0000FF' # Blue (Optimal NDWI)


# ==================================================================================
# --- HELPER FUNCTIONS FOR VISUALIZATION ---
# ==================================================================================

def load_all_nodes(gdf_water_nodes_path, gdf_land_nodes_path):
    """Loads and merges water and land nodes."""
    try:
        gdf_water_nodes = gpd.read_file(gdf_water_nodes_path)
        gdf_water_nodes['type'] = 'water'

        gdf_land_nodes = gpd.GeoDataFrame(geometry=[])
        if os.path.exists(gdf_land_nodes_path):
            gdf_land_nodes = gpd.read_file(gdf_land_nodes_path)
            gdf_land_nodes['type'] = 'land'
            if 'NDWI' not in gdf_land_nodes.columns: gdf_land_nodes['NDWI'] = NDWI_LAND_VALUE

        required_cols = ['NDWI', 'geometry', 'type']
        for gdf in [gdf_water_nodes, gdf_land_nodes]:
            if not gdf.empty:
                for col in required_cols:
                    if col not in gdf.columns:
                        # Filling missing columns for correct concatenation
                        gdf[col] = NDWI_LAND_VALUE if col == 'NDWI' and gdf['type'].iloc[0] == 'land' else None

        gdf_all_nodes = pd.concat([gdf_water_nodes[required_cols], gdf_land_nodes[required_cols]], ignore_index=True)
        gdf_all_nodes['node_id'] = range(len(gdf_all_nodes))

        return gdf_all_nodes
    except Exception as e:
        print(f"❌ Error loading GeoJSON nodes: {e}")
        return None

def create_path_geojson(path_nodes, gdf_all_nodes, color, name, weight=5, dash_array=''):
    """Creates a GeoJSON object for the path (LineString)."""
    path_geometries = [
        gdf_all_nodes[gdf_all_nodes['node_id'] == node_id].geometry.iloc[0]
        for node_id in path_nodes
        if node_id in gdf_all_nodes['node_id'].values
    ]

    if not path_geometries:
        return None

    path_line = LineString([(p.x, p.y) for p in path_geometries])
    gdf_path = gpd.GeoDataFrame([1], geometry=[path_line], crs=gdf_all_nodes.crs)

    return GeoJSON(
        data=gdf_path.__geo_interface__,
        style={'color': color, 'weight': weight, 'opacity': 0.9, 'dashArray': dash_array},
        name=name
    )

def plot_base_nodes(m, gdf_all_nodes):
    """Adds water nodes colored by NDWI and land markers to the map."""

    gdf_water_for_viz = gdf_all_nodes[gdf_all_nodes['type'] == 'water'].copy()

    if gdf_water_for_viz.empty:
        print("    ⚠️ Warning: No water nodes to display.")
        return

    # NDWI color logic
    min_ndwi = gdf_water_for_viz['NDWI'].min()
    max_ndwi = gdf_water_for_viz['NDWI'].max()
    ndwi_range = max_ndwi - min_ndwi
    cmap = plt.colormaps['RdYlGn']

    def get_color(ndwi_value):
        norm_ndwi = np.clip((ndwi_value - min_ndwi) / ndwi_range, 0, 1) if ndwi_range != 0 else 0.5
        rgba = cmap(norm_ndwi)
        return matplotlib.colors.rgb2hex(rgba[:3])

    gdf_water_for_viz['color'] = gdf_water_for_viz['NDWI'].apply(get_color)

    water_points_geojson = GeoJSON(
        data=gdf_water_for_viz.__geo_interface__,
        point_style={'radius': 3, 'fillOpacity': 0.9, 'weight': 0.5},
        style_callback=lambda feat: {'color': feat['properties']['color'], 'fillColor': feat['properties']['color']},
        name='NDWI Water Nodes'
    )
    m.add_layer(water_points_geojson)

    # Adding landmass (optional)
    gdf_land_for_viz = gdf_all_nodes[gdf_all_nodes['type'] == 'land']
    if not gdf_land_for_viz.empty:
        land_points_geojson = GeoJSON(
             data=gdf_land_for_viz.__geo_interface__,
             point_style={'radius': 3, 'fillOpacity': 0.5, 'weight': 0.5},
             style={'color': '#804000', 'fillColor': '#804000'},
             name='Land Nodes'
        )
        m.add_layer(land_points_geojson)


def plot_start_end_markers(m, gdf_all_nodes, boat_configs):
    """
    Adds Start/Finish markers for all routes.
    Icon colors correspond to the colors of the actual boat paths.
    """
    for config in boat_configs:
        start_id = config['start_node']
        end_id = config['end_node']
        boat_id = config['id']

        try:
            start_geom = gdf_all_nodes[gdf_all_nodes['node_id'] == start_id].geometry.iloc[0]
            end_geom = gdf_all_nodes[gdf_all_nodes['node_id'] == end_id].geometry.iloc[0]
        except IndexError:
            print(f"    ❌ Error: Start/Finish nodes for {boat_id} not found in gdf_all_nodes.")
            continue


        # Determine icon color based on boat ID
        marker_color = 'gray'
        if 'Shortest' in boat_id: marker_color = 'orange'
        elif 'Greenest' in boat_id: marker_color = 'purple'
        elif 'Chaotic' in boat_id: marker_color = 'pink'

        # Start Marker
        marker_icon_start = AwesomeIcon(name='forward', marker_color=marker_color, icon_color='white', prefix='fa')
        marker_start = Marker(location=(start_geom.y, start_geom.x), draggable=False, icon=marker_icon_start, title=f"START 🚀 {boat_id}")
        m.add_layer(marker_start)

        # Finish Marker
        marker_icon_end = AwesomeIcon(name='flag-checkered', marker_color=marker_color, icon_color='white', prefix='fa')
        marker_end = Marker(location=(end_geom.y, end_geom.x), draggable=False, icon=marker_icon_end, title=f"FINISH 🏁 {boat_id}")
        m.add_layer(marker_end)

def display_final_results_table(score_file_path):
    """
    Loads and displays the final results table (DataFrame) generated in Stage 3.
    """
    print("\n--- Final Simulation Results Table ---")
    try:
        with open(score_file_path, 'r') as f:
            final_scores = json.load(f)

        results_df = pd.DataFrame(final_scores).sort_values(by='score', ascending=False).reset_index(drop=True)
        results_df['rank'] = results_df.index + 1

        # Select and rename columns for clarity
        display_df = results_df[['rank', 'id', 'strategy', 'score', 'distance', 'avg_ndwi', 'steps', 'finished']]
        display_df.rename(columns={
            'id': 'Boat ID',
            'strategy': 'Strategy',
            'score': 'Total Score',
            'distance': 'Total Distance (deg)',
            'avg_ndwi': 'Average NDWI',
            'steps': 'Steps Taken',
            'finished': 'Finished'
        }, inplace=True)

        # Display the DataFrame as a formatted Markdown table
        print(f"✅ Results table loaded from: {score_file_path}")
        display(Markdown(display_df.to_markdown(index=False, floatfmt=".2f")))

    except FileNotFoundError:
        print(f"❌ Error: Results file not found at {score_file_path}. Please run 03_boat_simulator_and_scorer.py first.")
    except Exception as e:
        print(f"❌ Error loading or displaying results table: {e}")


# ==================================================================================
# --- MAIN VISUALIZATION FUNCTION ---
# ==================================================================================

def visualize_simulation_results():
    """Loads simulation data and displays results on a map."""
    print("==================================================================================")
    print("🗺️ STAGE 4: VISUALIZATION OF RESULTS (9 PATHS)")
    print("==================================================================================")

    # 1. Data Loading
    gdf_all_nodes = load_all_nodes(GEOJSON_NODES_FILE, GEOJSON_LAND_PIXELS_FILE)
    if gdf_all_nodes is None: return

    try:
        with open(BOAT_CONFIGS_FILE, 'r') as f:
            boat_configs = json.load(f)
        print(f"✅ Loaded configuration for {len(boat_configs)} boats.")
    except FileNotFoundError:
        print(f"❌ Error: Configuration file not found: {BOAT_CONFIGS_FILE}. Please run 02_graph_builder_and_pathfinder.py.")
        return
    except json.JSONDecodeError:
        print("❌ Error: Invalid JSON format in configuration file.")
        return

    try:
        with open(GRAPH_FILE, 'rb') as f:
            G = pickle.load(f)
        print(f"✅ Graph loaded: {G.number_of_nodes()} nodes.")
    except FileNotFoundError:
        print(f"❌ Error: Graph file not found: {GRAPH_FILE}. Please run 02_graph_builder_and_pathfinder.py.")
        return


    # 2. Map Initialization
    center_lat = gdf_all_nodes.geometry.y.mean()
    center_lon = gdf_all_nodes.geometry.x.mean()
    m = Map(center=(center_lat, center_lon), zoom=14, tiles=basemaps.CartoDB.DarkMatter)

    # 3. Display Base Elements
    print("⏳ Displaying NDWI nodes...")
    plot_base_nodes(m, gdf_all_nodes)
    plot_start_end_markers(m, gdf_all_nodes, boat_configs)

    # 4. Display Simulated and OPTIMAL (Proposed) Paths
    print("⏳ Displaying simulated and optimal paths (9 lines)...")

    # Using indexing for unique names in LayersControl
    boat_number_map = {'Shortest': '1', 'Greenest': '2', 'Chaotic': '3'}

    for config in boat_configs:
        path_nodes = config.get('path_nodes')
        boat_id = config['id']
        boat_type = boat_id.split('_')[-1] # Shortest, Greenest, Chaotic
        boat_num = boat_number_map.get(boat_type, 'X')

        # --- 4.1. Define SIMULATED Path Parameters ---
        color = None
        weight = 5
        weight_type = None

        if 'Shortest' in boat_id:
            color = COLOR_BOAT_1_ORANGE
            weight_type = 'weight_dist'
        elif 'Greenest' in boat_id:
            color = COLOR_BOAT_2_PURPLE
            weight_type = 'weight_ndwi'
        elif 'Chaotic' in boat_id:
            color = COLOR_BOAT_3_PINK
            weight = 3
            weight_type = 'weight_dist'

        if not color: continue


        # --- 4.2. RECALCULATE SIMULATED (ACTUAL) PATH ---
        # If no simulated path is in config, calculate it as the shortest path
        # using the corresponding weight (weight_type)
        if path_nodes is None or len(path_nodes) < 2:
            try:
                # Note: This path is calculated using the strategy's intended weight,
                # but it represents the 'actual' movement for visualization purposes.
                path_nodes = nx.shortest_path(G, config['start_node'], config['end_node'], weight=weight_type)
            except nx.NetworkXNoPath:
                print(f"    ❌ Path for '{boat_id}' not found, skipping.")
                continue

        # --- 4.3. DISPLAY SIMULATED (ACTUAL) PATH (3 lines) ---
        simulated_name = f"{boat_num}. SIMULATED '{boat_type}' (Bold)"
        if path_nodes and len(path_nodes) >= 2:
            path_geojson = create_path_geojson(path_nodes, gdf_all_nodes, color, simulated_name, weight=weight)
            if path_geojson:
                m.add_layer(path_geojson)
                print(f"    ✅ Route '{simulated_name}' added.")


        # --- 4.4. DISPLAY OPTIMAL PATHS (6 lines) ---

        # --- 4.4.1. Calculate and Display OPTIMAL Dist (RED DASHED) ---
        try:
            optimal_path_dist = nx.shortest_path(G, config['start_node'], config['end_node'], weight='weight_dist')

            optimal_name_dist = f"{boat_num}. OPTIMAL (Distance) for {boat_type}"
            optimal_geojson_dist = create_path_geojson(
                optimal_path_dist,
                gdf_all_nodes,
                COLOR_OPTIMAL_DIST_RED,
                optimal_name_dist,
                weight=1.5,
                dash_array='4, 4' # Dashed
            )

            if optimal_geojson_dist:
                m.add_layer(optimal_geojson_dist)
                print(f"    ⭐ Optimal Dist (Red dashed) added for {boat_type}.")

        except nx.NetworkXNoPath:
            print(f"    ❌ Optimal Dist path for '{boat_type}' not found.")

        # --- 4.4.2. Calculate and Display OPTIMAL NDWI (BLUE DASHED) ---
        try:
            optimal_path_ndwi = nx.shortest_path(G, config['start_node'], config['end_node'], weight='weight_ndwi')

            optimal_name_ndwi = f"{boat_num}. OPTIMAL (NDWI) for {boat_type}"
            optimal_geojson_ndwi = create_path_geojson(
                optimal_path_ndwi,
                gdf_all_nodes,
                COLOR_OPTIMAL_NDWI_BLUE,
                optimal_name_ndwi,
                weight=1.5,
                dash_array='4, 4' # Dashed
            )

            if optimal_geojson_ndwi:
                m.add_layer(optimal_geojson_ndwi)
                print(f"    ⭐ Optimal NDWI (Blue dashed) added for {boat_type}.")

        except nx.NetworkXNoPath:
            print(f"    ❌ Optimal NDWI path for '{boat_type}' not found.")

    # 5. Add Layers Control and Display Map
    from ipyleaflet import LayersControl
    m.add_control(LayersControl(position='topright'))

    print("\n✅ Map is ready. Total 9 routes displayed.")
    print("   1, 2, 3: Simulated Routes (Bold).")
    print("   1, 2, 3: Optimal Routes (Distance - Red Dashed).")
    print("   1, 2, 3: Optimal Routes (NDWI - Blue Dashed).")
    display(m)
    print("==================================================================================")

    # 6. Display Final Results Table
    display_final_results_table(SCORE_FILE)


if __name__ == '__main__':
    visualize_simulation_results()